### 1. Please access the http://oasis-deu.xyz:5002/ above and put the downloaded file in the `dataset` folder
### 2. `CUDA, cuDNN, nvidia driver` Please check the version
### 3. `pip install -r requirements.txt`

In [1]:
import tensorflow as tf

tf.config.list_physical_devices('GPU')

2022-06-05 15:21:53.903825: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
%cat dataset/data.yml

names:
- chilsung-cider
- coca-cola
- pepsi
- pocari-sweat
- sprite
- dr-pepper
- welchs-grape
- powerade
- gatorade
- monster-energy-ultra
nc: 10
train: /home/ml/workspace/2022-SolutionChallenge-CanDrink/ml/dataset/train.txt
val: /home/ml/workspace/2022-SolutionChallenge-CanDrink/ml/dataset/val.txt


In [3]:
from glob import glob

img_list = glob('dataset/export/images/*')

print(len(img_list))

2327


In [4]:
from sklearn.model_selection import train_test_split

train_img_list, val_img_list = train_test_split(img_list, test_size=0.2, random_state=2000)

print(len(train_img_list), len(val_img_list))

1861 466


In [5]:
with open('dataset/train.txt', 'w') as f:
  f.write('\n'.join(train_img_list) + '\n')

with open('dataset/val.txt', 'w') as f:
  f.write('\n'.join(val_img_list) + '\n')

In [6]:
import yaml
import os 

with open('dataset/data.yml', 'r') as f:
  data = yaml.load(f,Loader=yaml.FullLoader)

print(data)

data['train'] = f'{os.getcwd()}/dataset/train.txt'
data['val'] = f'{os.getcwd()}/dataset/val.txt'

with open('dataset/data.yml', 'w') as f:
  yaml.dump(data, f)

print(data)

{'names': ['chilsung-cider', 'coca-cola', 'pepsi', 'pocari-sweat', 'sprite', 'dr-pepper', 'welchs-grape', 'powerade', 'gatorade', 'monster-energy-ultra'], 'nc': 10, 'train': '/home/ml/workspace/2022-SolutionChallenge-CanDrink/ml/dataset/train.txt', 'val': '/home/ml/workspace/2022-SolutionChallenge-CanDrink/ml/dataset/val.txt'}
{'names': ['chilsung-cider', 'coca-cola', 'pepsi', 'pocari-sweat', 'sprite', 'dr-pepper', 'welchs-grape', 'powerade', 'gatorade', 'monster-energy-ultra'], 'nc': 10, 'train': '/home/ml/workspace/2022-SolutionChallenge-CanDrink/ml/dataset/train.txt', 'val': '/home/ml/workspace/2022-SolutionChallenge-CanDrink/ml/dataset/val.txt'}


In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import gc
import torch
gc.collect()
torch.cuda.empty_cache()

!python yolov5/train.py --img 1280 --batch 8 --epochs 50 --data dataset/data.yml --cfg yolov5/models/yolov5s.yaml --weights yolov5s.pt --name model818

train: weights=yolov5s.pt, cfg=yolov5/models/yolov5s.yaml, data=dataset/data.yml, hyp=yolov5/data/hyps/hyp.scratch-low.yaml, epochs=50, batch_size=8, imgsz=1280, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, noplots=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=yolov5/runs/train, name=model818, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: skipping check (offline), for updates see https://github.com/ultralytics/yolov5
requirements: tensorflow>=1.15.0 not found and is required by YOLOv5, attempting auto-update...
requirements: 'pip install tensorflow>=1.15.0' skipped (offline)
YOLOv5 🚀 v6.1-227-ga6e99e4 Python-3.8.10 torch-1.10.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3080 Ti, 12054MiB)

hyperparameters

In [9]:
!python yolov5/export.py --weights yolov5/runs/train/model8182/weights/best.pt --img 1280 --batch 1

export: data=yolov5/data/coco128.yaml, weights=['yolov5/runs/train/model8182/weights/best.pt'], imgsz=[1280], batch_size=1, device=cpu, half=False, inplace=False, train=False, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=12, verbose=False, workspace=4, nms=False, agnostic_nms=False, topk_per_class=100, topk_all=100, iou_thres=0.45, conf_thres=0.25, include=['torchscript', 'onnx']
YOLOv5 🚀 v6.1-227-ga6e99e4 Python-3.8.10 torch-1.10.0+cu113 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7037095 parameters, 0 gradients, 15.9 GFLOPs

PyTorch: starting from yolov5/runs/train/model8182/weights/best.pt with output shape (1, 100800, 15) (13.9 MB)

TorchScript: starting export with torch 1.10.0+cu113...
TorchScript: export success, saved as yolov5/runs/train/model8182/weights/best.torchscript (27.9 MB)

ONNX: starting export with onnx 1.11.0...
ONNX: export success, saved as yolov5/runs/train/model8182/weights/best.onnx (28.4 MB)

Export complete (3.76s)
Re

In [12]:
import onnx
from onnx_tf.backend import prepare

onnx_model = onnx.load("yolov5/runs/train/model8182/weights/best.onnx")  # load onnx model
tf_rep = prepare(onnx_model)  # prepare tf representation
tf_rep.export_graph("content/output/best9.pb")  # export the model

2022-06-05 16:09:37.598105: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-06-05 16:09:37.876429: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10101 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3080 Ti, pci bus id: 0000:01:00.0, compute capability: 8.6


INFO:tensorflow:Assets written to: content/output/best9.pb/assets


INFO:tensorflow:Assets written to: content/output/best9.pb/assets


In [13]:
import tensorflow as tf

saved_model_dir = 'content/output/best9.pb'
converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
converter.target_spec.supported_types = [tf.float16]
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()
open('converted_model-1280-f16-s.tflite', 'wb').write(tflite_model)

# import tensorflow as tf

# saved_model_dir = 'content/output/best5.pb'
# converter = tf.lite.TFLiteConverter.from_saved_model(saved_model_dir)
# converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
#                                        tf.lite.OpsSet.SELECT_TF_OPS]
# tflite_model = converter.convert()
# open('converted_model13.tflite', 'wb').write(tflite_model)

Estimated count of arithmetic ops: 69.710 G  ops, equivalently 34.855 G  MACs


2022-06-05 16:10:29.411953: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:362] Ignored output_format.
2022-06-05 16:10:29.411977: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:365] Ignored drop_control_dependency.
2022-06-05 16:10:29.412453: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: content/output/best9.pb
2022-06-05 16:10:29.418887: I tensorflow/cc/saved_model/reader.cc:81] Reading meta graph with tags { serve }
2022-06-05 16:10:29.418913: I tensorflow/cc/saved_model/reader.cc:122] Reading SavedModel debug info (if present) from: content/output/best9.pb
2022-06-05 16:10:29.435585: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:354] MLIR V1 optimization pass is not enabled
2022-06-05 16:10:29.437719: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-06-05 16:10:29.525654: I tensorflow/cc/saved_model/loader.cc:212] Running initialization op on SavedModel bundle at path: content/ou

15768876